In [47]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [48]:
from google.colab import drive

In [49]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
nfl_draft_raw = pd.read_csv('/content/drive/MyDrive/nfl_draft_2014_2024.csv', index_col=False)
college_passing_raw = pd.read_csv('/content/drive/MyDrive/college_passing_data_v3.csv', index_col=False)
college_rushing_raw = pd.read_csv('/content/drive/MyDrive/college_rushing_data_v3.csv', index_col=False)
college_receiving_raw = pd.read_csv('/content/drive/MyDrive/college_receiving_data_v3.csv', index_col=False)


In [51]:
# Re-label Conference and Division values for each player to be the most recent/last conference they played in #
college_passing_raw['Season'] = college_passing_raw['Season'].astype(int)
college_rushing_raw['Season'] = college_rushing_raw['Season'].astype(int)
college_receiving_raw['Season'] = college_receiving_raw['Season'].astype(int)

In [52]:
# Group by 'Player' and get the row with maximum 'Season' for each player
max_season_index_passing = college_passing_raw.groupby('Player')['Season'].idxmax()
max_season_index_rushing = college_rushing_raw.groupby('Player')['Season'].idxmax()
max_season_index_receiving = college_receiving_raw.groupby('Player')['Season'].idxmax()

In [53]:
# Create a dictionary to map player names to their most recent conferences and divisions
player_conference_map_passing = college_passing_raw.loc[max_season_index_passing].set_index('Player')['Conference'].to_dict()
player_division_map_passing = college_passing_raw.loc[max_season_index_passing].set_index('Player')['Division'].to_dict()
player_power5_map_passing = college_passing_raw.loc[max_season_index_passing].set_index('Player')['Power5'].to_dict()
player_school_map_passing = college_passing_raw.loc[max_season_index_passing].set_index('Player')['School'].to_dict()

player_conference_map_rushing = college_rushing_raw.loc[max_season_index_rushing].set_index('Player')['Conference'].to_dict()
player_division_map_rushing = college_rushing_raw.loc[max_season_index_rushing].set_index('Player')['Division'].to_dict()
player_power5_map_rushing = college_rushing_raw.loc[max_season_index_rushing].set_index('Player')['Power5'].to_dict()
player_school_map_rushing = college_rushing_raw.loc[max_season_index_rushing].set_index('Player')['School'].to_dict()

player_conference_map_receiving = college_receiving_raw.loc[max_season_index_receiving].set_index('Player')['Conference'].to_dict()
player_division_map_receiving = college_receiving_raw.loc[max_season_index_receiving].set_index('Player')['Division'].to_dict()
player_power5_map_receiving = college_receiving_raw.loc[max_season_index_receiving].set_index('Player')['Power5'].to_dict()
player_school_map_receiving = college_receiving_raw.loc[max_season_index_receiving].set_index('Player')['School'].to_dict()


In [54]:
# Update 'Conference' and 'Division' columns based on the mapping
college_passing_raw['Conference'] = college_passing_raw['Player'].map(player_conference_map_passing)
college_passing_raw['Division'] = college_passing_raw['Player'].map(player_division_map_passing)
college_passing_raw['Power5'] = college_passing_raw['Player'].map(player_power5_map_passing)
college_passing_raw['School'] = college_passing_raw['Player'].map(player_school_map_passing)

college_rushing_raw['Conference'] = college_rushing_raw['Player'].map(player_conference_map_rushing)
college_rushing_raw['Division'] = college_rushing_raw['Player'].map(player_division_map_rushing)
college_rushing_raw['Power5'] = college_rushing_raw['Player'].map(player_power5_map_rushing)
college_rushing_raw['School'] = college_rushing_raw['Player'].map(player_school_map_rushing)

college_receiving_raw['Conference'] = college_receiving_raw['Player'].map(player_conference_map_receiving)
college_receiving_raw['Division'] = college_receiving_raw['Player'].map(player_division_map_receiving)
college_receiving_raw['Power5'] = college_receiving_raw['Player'].map(player_power5_map_receiving)
college_receiving_raw['School'] = college_receiving_raw['Player'].map(player_school_map_receiving)


In [55]:
display(college_passing_raw)
display(college_rushing_raw)
display(college_receiving_raw)

,Season,Rank,Player,School,Power5,Conference,Division,Cl,Pos,G,Pass Att,Pass Com,Int,Pass TD,Pass Yds
0,2013,108,A.J. Augustine,VMI,N,Big South,FCS,Sr.,QB,11,158,93,7,9,1149
1,2013,116,A.J. Doyle,Massachusetts,N,MAC,FBS,So.,QB,11,235,128,11,6,1274
2,2013,30,A.J. McCarron,Alabama,Y,SEC,FBS,Sr.,QB,13,336,226,7,28,3063
3,2014,123,A.J. Schurr,Army West Point,N,FBS Independent,FBS,Jr.,QB,10,28,12,1,1,242
4,2023,37,Aaron Allen,Alcorn,N,SWAC,FCS,Sr.,QB,11,302,193,6,15,2323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,2021,37,Zevi Eckhaus,Bryant,N,OVC,FCS,Fr.,QB,11,370,232,3,21,2378
3026,2022,114,Zion Turner,UConn,N,FBS Independent,FBS,Fr.,QB,13,258,149,10,9,1407
3027,2020,10,Zion Webb,Jacksonville St.,N,ASUN,FCS,Jr.,QB,12,205,120,8,13,1777
3028,2022,72,Zion Webb,Jacksonville St.,N,ASUN,FCS,Sr.,QB,11,201,111,9,10,1737


,Season,Rank,Player,School,Power5,Conference,Division,Cl,Pos,G,Rush,Rush TD,Rush Yds
0,2014,89,A. Baker,Alcorn,N,SWAC,FCS,---,NaN,13,113,13,725
1,2020,179,A.J. Carter,McNeese,N,Southland,FCS,So.,RB,4,31,2,156
2,2019,188,A.J. Davis,Pittsburgh,Y,ACC,FBS,Jr.,RB,12,127,4,530
3,2018,3,A.J. Hines,Duquesne,N,NEC,FCS,Jr.,RB,12,266,15,1520
4,2016,10,A.J. Hines,Duquesne,N,NEC,FCS,Fr.,RB,11,242,13,1291
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4602,2020,30,Zonovan Knight,NC State,Y,ACC,FBS,So.,RB,12,143,10,788
4603,2021,90,Zonovan Knight,NC State,Y,ACC,FBS,So.,RB,12,140,3,753
4604,2019,104,Zonovan Knight,NC State,Y,ACC,FBS,Fr.,RB,12,136,5,745
4605,2022,170,Zuberi Mobley,Fla. Atlantic,N,CUSA,FBS,So.,RB,12,101,3,534


,Season,Rank,Player,School,Power5,Conference,Division,Cl,Pos,G,Rec,Rec Yds,Rec TD
0,2013,204.0,A.C. Leonard,Tennessee St.,N,OVC,FCS,Jr.,TE,12,34,441,5
1,2018,7.0,A.J. Brown,Ole Miss,Y,SEC,FBS,Jr.,WR,12,85,1320,6
2,2017,10.0,A.J. Brown,Ole Miss,Y,SEC,FBS,So.,WR,12,75,1252,11
3,2017,186.0,A.J. Coney,Akron,N,MAC,FBS,Sr.,WR,14,43,551,3
4,2014,107.0,A.J. Johnson,Indiana St.,N,MVFC,FCS,Sr.,WR,12,48,633,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4515,2014,NaN,Ben Koyack,Notre Dame,N,FBS Independent,FBS,Sr.,TE,13,30,317,2
4516,2014,NaN,Geoff Swaim,Texas,Y,Big 12,FBS,Sr.,TE,13,10,70,1
4517,2013,NaN,Rory Anderson,South Carolina,Y,SEC,FBS,Jr.,TE,12,17,235,0
4518,2014,NaN,Rory Anderson,South Carolina,Y,SEC,FBS,Sr.,TE,10,22,260,1


In [56]:
## Data Preparation ##

# Establish copies #
nfl_draft = nfl_draft_raw.copy()
college_passing = college_passing_raw.copy()
college_rushing = college_rushing_raw.copy()
college_receiving = college_receiving_raw.copy()

# Remove Freshman from the 2023 season as they are not eligible for this upcoming draft (or any draft) #
# It is okay to leave Freshman in for every other season because they will have been eligible for the NFL draft at some point #
#(Freshman in 2022 -> not eligible for draft in 2022, but is eligible for draft in 2023 when they are a sophomore)#
# If I don't remove these players it will impact the results of the model (i.e. A freshman could have an awesome season, but he will still be labeled as undrafted) #
# NFL Draft rules state a player must be out of high school for at least 3 years before being eligible to be drafted, the data collected does not list Red Shirt seasons, so #
# for simplicity, I will just exclude players labeled as freshman. This is not great, but it is the only way I have currently thought of to do this #
college_passing = college_passing[~((college_passing['Season'] == 2023) & (college_passing['Cl'] == 'Fr.'))]
college_rushing = college_rushing[~((college_rushing['Season'] == 2023) & (college_rushing['Cl'] == 'Fr.'))]
college_receiving = college_receiving[~((college_receiving['Season'] == 2023) & (college_receiving['Cl'] == 'Fr.'))]

# Drop unnecessary columns from datasets #
nfl_draft = nfl_draft[['DraftYear','Pos','Pick','Player']]
college_passing = college_passing[['Player','Pos','Division','Conference','Power5','School','G','Pass Com','Pass Att','Pass Yds','Pass TD','Int']]
college_rushing = college_rushing[['Player','Pos','Division','Conference','Power5','School','G','Rush','Rush Yds','Rush TD']]
college_receiving = college_receiving[['Player','Pos','Division','Conference','Power5','School','G','Rec','Rec Yds','Rec TD']]

# Manipulate/Create new variable for each position to indicate which number at the respective position they were drafted #
quarterback_draft = nfl_draft[nfl_draft['Pos']=='QB'].copy()
quarterback_draft['position_pick_number'] = quarterback_draft.groupby('DraftYear')['Pick'].rank().astype(int)
quarterback_draft = quarterback_draft[['Player','position_pick_number']]

running_back_draft = nfl_draft[nfl_draft['Pos']=='RB'].copy()
running_back_draft['position_pick_number'] = running_back_draft.groupby('DraftYear')['Pick'].rank().astype(int)
running_back_draft = running_back_draft[['Player','position_pick_number']]

wide_receiver_draft = nfl_draft[nfl_draft['Pos']=='WR'].copy()
wide_receiver_draft['position_pick_number'] = wide_receiver_draft.groupby('DraftYear')['Pick'].rank().astype(int)
wide_receiver_draft = wide_receiver_draft[['Player','position_pick_number']]

tight_end_draft = nfl_draft[nfl_draft['Pos']=='TE'].copy()
tight_end_draft['position_pick_number'] = tight_end_draft.groupby('DraftYear')['Pick'].rank().astype(int)
tight_end_draft = tight_end_draft[['Player','position_pick_number']]

# display(college_passing)
# display(college_rushing)
# display(college_receiving)
# display(quarterback_draft)
# display(running_back_draft)
# display(wide_receiver_draft)
# display(tight_end_draft)

In [57]:
## Data Preparation ##

# Aggregate player data to get career stats instead of individual season stats #

# Quarterback Passing #
career_stats_qb_passing = college_passing.copy()
career_stats_qb_passing = career_stats_qb_passing[career_stats_qb_passing['Pos']=='QB']
career_stats_qb_passing = career_stats_qb_passing.groupby(['Player','Division','Conference','Power5','School']).sum().reset_index()
career_stats_qb_passing['Pass_Yds/Att'] = (career_stats_qb_passing['Pass Yds'] / career_stats_qb_passing['Pass Att']).round(1)
career_stats_qb_passing['Pass_Yds/G'] = (career_stats_qb_passing['Pass Yds'] / career_stats_qb_passing['G']).round(1)
career_stats_qb_passing['Pass_Com/G'] = (career_stats_qb_passing['Pass Com'] / career_stats_qb_passing['G']).round(1)
career_stats_qb_passing['Pass_TD/G'] = (career_stats_qb_passing['Pass TD'] / career_stats_qb_passing['G']).round(1)
career_stats_qb_passing['Int/G'] = (career_stats_qb_passing['Int'] / career_stats_qb_passing['G']).round(1)

# Running Back Rushing #
career_stats_rb_rushing = college_rushing.copy()
career_stats_rb_rushing = career_stats_rb_rushing[career_stats_rb_rushing['Pos']=='RB']
career_stats_rb_rushing = career_stats_rb_rushing.groupby(['Player','Division','Conference','Power5','School']).sum().reset_index()
career_stats_rb_rushing['Rush Yds/Att'] = (career_stats_rb_rushing['Rush Yds'] / career_stats_rb_rushing['Rush']).round(1)
career_stats_rb_rushing['Rush Yds/G'] = (career_stats_rb_rushing['Rush Yds'] / career_stats_rb_rushing['G']).round(1)
career_stats_rb_rushing['Rush TD/G'] = (career_stats_rb_rushing['Rush TD'] / career_stats_rb_rushing['G']).round(1)

# Wide Receiver Receiving #
career_stats_wr_receiving = college_receiving.copy()
career_stats_wr_receiving = career_stats_wr_receiving[career_stats_wr_receiving['Pos']=='WR']
career_stats_wr_receiving = career_stats_wr_receiving.groupby(['Player','Division','Conference','Power5','School']).sum().reset_index()
career_stats_wr_receiving['Rec Yds/Rec'] = (career_stats_wr_receiving['Rec Yds'] / career_stats_wr_receiving['Rec']).round(1)
career_stats_wr_receiving['Rec Yds/G'] = (career_stats_wr_receiving['Rec Yds'] / career_stats_wr_receiving['G']).round(1)
career_stats_wr_receiving['Rec TD/G'] = (career_stats_wr_receiving['Rec TD'] / career_stats_wr_receiving['G']).round(1)


# Tight End Receiving #
career_stats_te_receiving = college_receiving.copy()
career_stats_te_receiving = career_stats_te_receiving[career_stats_te_receiving['Pos']=='TE']
career_stats_te_receiving = career_stats_te_receiving.groupby(['Player','Division','Conference','Power5','School']).sum().reset_index()
career_stats_te_receiving['Rec Yds/Rec'] = (career_stats_te_receiving['Rec Yds'] / career_stats_te_receiving['Rec']).round(1)
career_stats_te_receiving['Rec Yds/G'] = (career_stats_te_receiving['Rec Yds'] / career_stats_te_receiving['G']).round(1)
career_stats_te_receiving['Rec TD/G'] = (career_stats_te_receiving['Rec TD'] / career_stats_te_receiving['G']).round(1)

# display(career_stats_qb_passing)
# display(career_stats_rb_rushing)
# display(career_stats_wr_receiving)
# display(career_stats_te_receiving)

In [58]:
## Merge Performance dataframes with Draft dataframes ##

final_qb_data = pd.merge(career_stats_qb_passing, quarterback_draft,on='Player',how='left')
final_rb_data = pd.merge(career_stats_rb_rushing, running_back_draft,on='Player',how='left')
final_wr_data = pd.merge(career_stats_wr_receiving, wide_receiver_draft,on='Player',how='left')
final_te_data = pd.merge(career_stats_te_receiving, tight_end_draft,on='Player',how='left')

# Create function to determine if a player is drafted or not #
def drafted_or_not(value):
    if np.isnan(value):
        return 'N'
    else:
        return 'Y'

# Add a new column "Drafted?" based on the condition #
final_qb_data['Drafted?'] = final_qb_data['position_pick_number'].apply(drafted_or_not)
final_rb_data['Drafted?'] = final_rb_data['position_pick_number'].apply(drafted_or_not)
final_wr_data['Drafted?'] = final_wr_data['position_pick_number'].apply(drafted_or_not)
final_te_data['Drafted?'] = final_te_data['position_pick_number'].apply(drafted_or_not)

# Fill NaN values in position_pick_number #
final_qb_data['position_pick_number'] = final_qb_data['position_pick_number'].fillna('Undrafted')
final_rb_data['position_pick_number'] = final_rb_data['position_pick_number'].fillna('Undrafted')
final_wr_data['position_pick_number'] = final_wr_data['position_pick_number'].fillna('Undrafted')
final_te_data['position_pick_number'] = final_te_data['position_pick_number'].fillna('Undrafted')

# Rearrange columns #
final_qb_data = final_qb_data[['Player','Division','Conference','Power5','School','Drafted?','position_pick_number','G','Pass Com','Pass Att','Pass Yds','Pass TD','Int','Pass_Yds/Att','Pass_Com/G','Pass_Yds/G',
                               'Pass_TD/G','Int/G']]
final_rb_data = final_rb_data[['Player','Division','Conference','Power5','School','Drafted?','position_pick_number','G','Rush','Rush Yds','Rush TD','Rush Yds/Att','Rush Yds/G','Rush TD/G']]
final_wr_data = final_wr_data[['Player','Division','Conference','Power5','School','Drafted?','position_pick_number','G','Rec','Rec Yds','Rec TD','Rec Yds/Rec','Rec Yds/G','Rec TD/G']]
final_te_data = final_te_data[['Player','Division','Conference','Power5','School','Drafted?','position_pick_number','G','Rec','Rec Yds','Rec TD','Rec Yds/Rec','Rec Yds/G','Rec TD/G']]


In [127]:
###Quarterback GB Model###

# Label Encoding for 'Division' and 'Power5 columns
label_encoder = LabelEncoder()
final_qb_data['Division'] = label_encoder.fit_transform(final_qb_data['Division'])
final_qb_data['Power5'] = label_encoder.fit_transform(final_qb_data['Power5'])

# Establish X and y dataframes for the model #
X_quarterback = final_qb_data[['Power5','Division','Pass_Com/G','Pass_Yds/Att','Pass_Yds/G','Pass_TD/G','Int/G']]
y_quarterback = final_qb_data['Drafted?']
# Split the data into a training set (80%) and a testing set (20%)
X_train_quarterback, X_test_quarterback, Y_train_quarterback, Y_test_quarterback = train_test_split(X_quarterback, y_quarterback, test_size=0.15, random_state=23)


In [128]:
#Set model
model = GradientBoostingClassifier()

In [129]:
#fit model to training data set
model.fit(X_train_quarterback,Y_train_quarterback)

GradientBoostingClassifier()

In [130]:
#calculate Cross Validation Accuracy Score
cross_val_score(model,X_train_quarterback,Y_train_quarterback,cv=5, n_jobs = -1).mean()

0.9237354085603113

In [131]:
#Establish Parameters as this will help us find best numbers to calculate best CV score
param_grid = {
    'n_estimators':[10,50,100,500],
    'learning_rate':[0.0001,.001,.01,.1,1.0],
    'max_depth':[3,5,7,9]

}

In [132]:
#Use new parameters with GridSearchCV model
gbr = GridSearchCV(model,param_grid,cv=5,n_jobs=-1)

In [133]:
#fit the model with the training data
gbr.fit(X_train_quarterback,Y_train_quarterback)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [10, 50, 100, 500]})

In [134]:
#Find best parameters
gbr.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [135]:
#Find best score
gbr.best_score_

0.9276264591439689

In [136]:
###Runningback GB Model###

label_encoder = LabelEncoder()
final_rb_data['Division'] = label_encoder.fit_transform(final_rb_data['Division'])
final_rb_data['Power5'] = label_encoder.fit_transform(final_rb_data['Power5'])
# Establish X and y dataframes for the model #
X_runningback = final_rb_data[['Power5','Division','Rush Yds/Att','Rush Yds/G','Rush TD/G']]
y_runningback = final_rb_data['Drafted?']
# Split the data into a training set (80%) and a testing set (20%)
X_train_runningback, X_test_runningback, Y_train_runningback, Y_test_runningback = train_test_split(X_runningback, y_runningback, test_size=0.2, random_state=23)

In [137]:
#fit model to training data set
model.fit(X_train_runningback,Y_train_runningback)

GradientBoostingClassifier()

In [138]:
#calculate Cross Validation Accuracy Score
cross_val_score(model,X_train_runningback,Y_train_runningback,cv=5, n_jobs = -1).mean()

0.9182394762416244

In [139]:
#Establish Parameters as this will help us find best numbers to calculate best CV score
param_grid = {
    'n_estimators':[10,50,100,500],
    'learning_rate':[0.0001,.001,.01,.1,1.0],
    'max_depth':[3,5,7,9]

}

In [140]:
#Use new parameters with GridSearchCV model
gbr = GridSearchCV(model,param_grid,cv=5,n_jobs=-1)

In [141]:
#fit the model with the training data
gbr.fit(X_train_runningback,Y_train_runningback)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [10, 50, 100, 500]})

In [144]:
#Find best parameters
gbr.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}

In [145]:
#Find best parameters
gbr.best_score_

0.9264146761461477

In [152]:
###Wide Receiver GB Model###

label_encoder = LabelEncoder()
final_wr_data['Division'] = label_encoder.fit_transform(final_wr_data['Division'])
final_wr_data['Power5'] = label_encoder.fit_transform(final_wr_data['Power5'])
# Establish X and y dataframes for the model #
X_widereceiver = final_wr_data[['Division','Power5','Rec Yds/Rec','Rec Yds/G','Rec TD/G']]
y_widereceiver = final_wr_data['Drafted?']
# Split the data into a training set (80%) and a testing set (20%)
X_train_widereceiver, X_test_widereceiver, Y_train_widereceiver, Y_test_widereceiver = train_test_split(X_widereceiver, y_widereceiver, test_size=0.2, random_state=23)


In [153]:
#fit model to training data set
model.fit(X_train_widereceiver,Y_train_widereceiver)

GradientBoostingClassifier()

In [154]:
#calculate Cross Validation Accuracy Score
cross_val_score(model,X_train_widereceiver,Y_train_widereceiver,cv=5, n_jobs = -1).mean()

0.8838694339137698

In [155]:
#Establish Parameters as this will help us find best numbers to calculate best CV score
param_grid = {
    'n_estimators':[10,50,100,500],
    'learning_rate':[0.0001,.001,.01,.1,1.0],
    'max_depth':[3,5,7,9]

}

In [156]:
#Use new parameters with GridSearchCV model
gbr = GridSearchCV(model,param_grid,cv=5,n_jobs=-1)

In [157]:
#fit the model with the training data
gbr.fit(X_train_widereceiver,Y_train_widereceiver)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [10, 50, 100, 500]})

In [158]:
#Find best parameters
gbr.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}

In [159]:
#Find best parameters
gbr.best_score_

0.8909721776007196

In [160]:
###Tight End GB Model###

label_encoder = LabelEncoder()
final_te_data['Division'] = label_encoder.fit_transform(final_te_data['Division'])
final_te_data['Power5'] = label_encoder.fit_transform(final_te_data['Power5'])
# Establish X and y dataframes for the model #
X_tightend = final_te_data[['Division','Power5','Rec Yds/Rec','Rec Yds/G','Rec TD/G']]
y_tightend = final_te_data['Drafted?']
# Split the data into a training set (80%) and a testing set (20%)
X_train_tightend, X_test_tightend, Y_train_tightend, Y_test_tightend = train_test_split(X_tightend, y_tightend, test_size=0.2, random_state=23)



In [161]:
#fit model to training data set
model.fit(X_train_tightend,Y_train_tightend)

GradientBoostingClassifier()

In [162]:
#calculate Cross Validation Accuracy Score
cross_val_score(model,X_train_tightend,Y_train_tightend,cv=5, n_jobs = -1).mean()

0.6358156028368794

In [163]:
#Establish Parameters as this will help us find best numbers to calculate best CV score
param_grid = {
    'n_estimators':[10,50,100,500],
    'learning_rate':[0.0001,.001,.01,.1,1.0],
    'max_depth':[3,5,7,9]

}

In [164]:
#Use new parameters with GridSearchCV model
gbr = GridSearchCV(model,param_grid,cv=5,n_jobs=-1)

In [168]:
#fit the model with the training data
gbr.fit(X_train_tightend,Y_train_tightend)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [10, 50, 100, 500]})

In [169]:
#Find best parameters
gbr.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 10}

In [170]:
#Find best parameters
gbr.best_score_

0.716223404255319